# **Sprint 8**
## 5. Análisis de duración de viajes — sábados (Loop --> O'Hare)

Este cuaderno implementa un flujo **limpio, modular y reproducible** para analizar la duración de los viajes en taxi desde el Loop hacia el Aeropuerto O'Hare, restringido a los **sábados**.  

El objetivo es doble:  
1. **Diagnóstico exploratorio**: caracterizar la distribución de la duración y su relación con la condición climática (lluvioso vs. sin lluvia).  
2. **Inferencia robusta**: aplicar pruebas estadísticas y tamaños de efecto que permitan decidir si la diferencia observada es significativa y relevante en la práctica.

---

### **Estructura del notebook**
1. **Configuración de parámetros**: define semillas, umbrales y estética de gráficos para asegurar reproducibilidad.  
2. **Carga de datos**: ingesta segura y validación inicial de columnas clave.  
3. **Utilidades**: funciones auxiliares para formateo, normalidad, tamaños de efecto e intervalos de confianza bootstrap.  
4. **Limpieza y validación**: reglas realistas (plausibilidad, IQR, nulos), garantizando un dataset confiable.  
5. **Análisis Exploratorio de Datos (AED)**: métricas descriptivas, tendencias temporales y distribuciones por clima.  
6. **Supuestos y pruebas**: árbol de decisión para seleccionar la prueba adecuada (t/Welch o Mann–Whitney) y cálculo de tamaños de efecto.  
7. **Comparación visual final**: violín + box + jitter resaltando media/mediana para comunicar diferencias de manera intuitiva.  
8. **Conclusiones**: salida compacta en Markdown con interpretación estadística, magnitud práctica e implicaciones accionables.

---

### **Valor esperado**
Este sprint no solo busca determinar si existe una diferencia estadísticamente significativa en la duración de los viajes según el clima, sino también **cuantificar su magnitud práctica**, **evaluar su robustez** y **generar insumos claros para la toma de decisiones operativas y de comunicación**.

## 1. Parámetros globales y carga de datos

### **Objetivo**  
Definir parámetros que gobiernan el análisis y la reproducibilidad, cargar el CSV con la columna temporal correctamente parseada y preparar la estética de los gráficos junto con formateadores numéricos en estilo es-MX.

### **Qué hace este bloque y por qué**  
- **Opciones de visualización**: configura `pandas` para mostrar todas las columnas y ampliar el ancho de salida, facilitando la inspección de los datos en Jupyter.  
- **Parámetros del flujo**: se fijan valores globales como:  
  - `ALFA`: nivel de significancia para pruebas estadísticas.  
  - `GRAF` y `EDA`: switches para controlar si se grafican métricas y se imprimen resultados descriptivos.  
  - `NBOOT`: número de réplicas bootstrap para análisis no paramétricos.  
  - `RM_OUT`: decisión sobre remover atípicos mediante IQR.  
  - `MIN_S` y `MAX_S`: umbrales realistas de duración (20 a 90 min).  
  - `RNG_SEED`: semilla para reproducibilidad en cálculos aleatorios.  
- **Carga robusta**: lectura del archivo `project_sql_result_07.csv` con `try/except`, parseando `start_ts` a `datetime` para habilitar operaciones temporales. Se imprime la cantidad de filas y las primeras observaciones. Si falla, se inicializa `df` vacío y se muestra el error.  
- **Estética unificada**: `matplotlib.rcParams` se actualiza para asegurar consistencia visual en todas las gráficas (tamaño, grilla tenue, tipografías).  
- **Formateadores numéricos es-MX**:  
  - `fmt_miles_es`: para formatear ticks en ejes (`FuncFormatter`).  
  - `str_miles_es`: para etiquetas de valores en barras.  
  Ambos utilizan separador de miles con punto y coma decimal, siguiendo la convención local.

### **Supuestos y alcance de este paso**  
- Solo se parsea `start_ts`, aún no se renombran columnas ni se aplican filtros de límites o outliers.  
- El pipeline posterior debe validar que `df` no esté vacío si la carga falla.

### **Resultados esperados**  
- `df` cargado con `start_ts` en formato `datetime64[ns]`.  
- Vista preliminar de las primeras filas.  
- Estética de gráficos y formateadores numéricos listos para usarse en los análisis posteriores.  


In [ ]:
# --- Parámetros globales y semillas — controlan todo el flujo ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from matplotlib.ticker import FuncFormatter
from IPython.display import display, Markdown

# Mostrar dataframes cómodamente en Jupyter (todas las columnas, ancho mayor)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

# Hiperparámetros del análisis
ALFA    = 0.05   # Nivel de significancia para pruebas estadísticas
GRAF    = True   # Mostrar (True) u ocultar (False) gráficas
EDA     = True   # Imprimir (True) u omitir (False) métricas descriptivas
NBOOT   = 9999   # Réplicas bootstrap (si se usan más adelante)
RM_OUT  = True   # Remover atípicos por IQR (si se aplica después en el flujo)
MIN_S   = 1200   # Límite inferior realista de duración en segundos (20 min)
MAX_S   = 5400   # Límite superior realista de duración en segundos (1.5 h)
RNG_SEED = 12345 # Semilla para reproducibilidad

# (Opcional) Fijar semilla para cualquier rutina aleatoria posterior
# np.random.seed(RNG_SEED)

# --- Carga de datos ---
try:
    # Lee el CSV y parsea 'start_ts' como datetime para habilitar operaciones .dt
    df = pd.read_csv('../data/project_sql_result_07.csv', parse_dates=['start_ts'])
    print("Datos cargados con éxito. Filas:", len(df))
    display(df.head(5))
except Exception as e:
    # Si falla la carga, dejamos df vacío (el resto del pipeline debe validar esto)
    print("Error cargando datos:", e)
    df = pd.DataFrame()
    

# --- Estética consistente de gráficos ---
plt.rcParams.update({
    "figure.figsize": (11, 5.5),   # tamaño por defecto de figuras
    "figure.dpi": 120,             # resolución
    "axes.grid": True,             # activar grilla suave
    "grid.alpha": 0.6,
    "grid.linewidth": 0.5,
    "axes.spines.top": False,      # quitar bordes superior/derecho
    "axes.spines.right": False,
    "axes.titlesize": 12,
    "axes.labelsize": 11,
    "xtick.labelsize": 10,
    "ytick.labelsize": 10,
})

def fmt_miles_es(x, pos=None):
    """
    Devuelve un string para formatear valores numéricos en ejes (es-MX).
    
    Reglas:
    - |x| < 1000 → un decimal.
    - Si no, entero con separador de miles.
    - Intercambia coma/punto para estándar español (1.234,5).
    
    Parámetros
    ----------
    x : float
        Valor numérico a formatear (lo provee Matplotlib).
    pos : int, opcional
        Índice del tick (no se usa; está para compatibilidad con FuncFormatter).
    
    Retorna
    -------
    str
        Número formateado con convención es-MX.
    """
    if pd.isna(x):
        return ""
    s = f"{x:,.1f}" if abs(x) < 1000 else f"{int(round(x)):,}"
    return s.replace(",", "_").replace(".", ",").replace("_", ".")

def str_miles_es(x, dec=1):
    """
    Devuelve un string formateado (es-MX) para anotar valores dentro de las gráficas.
    
    Parámetros
    ----------
    x : float or int
        Valor a formatear.
    dec : int, por defecto 1
        Decimales cuando |x| < 1000.
    
    Retorna
    -------
    str
        Representación en string con separador de miles y coma decimal.
    """
    if pd.isna(x):
        return ""
    if isinstance(x, float) and abs(x) < 1000:
        s = f"{x:.{dec}f}"
    else:
        s = f"{int(round(x)):,}"
    return s.replace(",", "_").replace(".", ",").replace("_", ".")


## 2. Renombrado, normalización y filtrado inicial

### **Objetivo**  
Estandarizar el esquema de columnas y la codificación de clima; asegurar tipos correctos; aplicar un filtro de plausibilidad de duración y (opcionalmente) remover atípicos por IQR, preparando un dataset consistente para el EDA y las pruebas.

### **Qué hace este bloque y por qué**  
- **Estandarización de columnas** (`ren`): unifica nombres (`inicio_ts`, `clima`, `duracion_s`) y hace cast robusto a `datetime`/numérico.  
- **Normalización de clima**: mapea variantes comunes (`bad/good`, `rainy/clear`) a dos niveles canónicos: `lluvioso` y `sin lluvia`.  
- **Plausibilidad de duración**: aplica límites realistas (`MIN_S=1200`, `MAX_S=5400`) para descartar registros imposibles o no representativos.  
- **Atípicos por grupo (opcional)**: si `RM_OUT=True`, elimina outliers por la regla IQR **dentro de cada categoría de clima**, evitando sesgar comparaciones.

### **Supuestos y alcance**  
- Los datos contienen, al menos, `inicio_ts`, `clima` y `duracion_s` tras el renombrado.  
- La normalización de `clima` cubre las variantes esperadas; valores desconocidos se asignan a `sin lluvia`.  
- La limpieza por IQR es **diagnóstica** y debe reportarse cuando se emplea (impacto en N y distribución).

### **Resultados esperados**  
- `df_filtrado` con esquema y tipos consistentes, sin duraciones fuera de rango, y con outliers removidos (si corresponde).  
- Un conteo claro de filas resultantes para trazar el linaje de datos.  

In [ ]:
# --- Renombrado y normalización (bad/good -> lluvioso/sin lluvia) ---
def ren(df: pd.DataFrame) -> pd.DataFrame:
    """
    Estandariza nombres de columnas y normaliza la variable categórica de clima.

    - Renombra columnas posibles del CSV a un esquema consistente:
      * start_ts / inicio_ts -> 'inicio_ts' (datetime)
      * weather_conditions / condiciones_climaticas -> 'clima' (str normalizada)
      * duration_seconds / duracion_segundos -> 'duracion_s' (num)
    - Convierte tipos: 'inicio_ts' a datetime y 'duracion_s' a numérico.
    - Normaliza 'clima' a {'lluvioso', 'sin lluvia'} mapeando variantes comunes.

    Parámetros
    ----------
    df : pd.DataFrame
        Datos crudos tras la lectura del CSV.

    Retorna
    -------
    pd.DataFrame
        Copia del DataFrame con columnas estandarizadas y 'clima' normalizado.

    Lanza
    -----
    ValueError
        Si faltan columnas requeridas tras el renombrado.
    """
    # Mapeo flexible de nombres de columnas -> estándar interno
    m = {
        'start_ts': 'inicio_ts',
        'weather_conditions': 'clima',
        'duration_seconds': 'duracion_s',
        'duracion_segundos': 'duracion_s',
        'condiciones_climaticas': 'clima',
    }
    # Renombra solo las que existan, y trabaja sobre una copia
    df = df.rename(columns={k: v for k, v in m.items() if k in df.columns}).copy()

    # Validación de esquema mínimo
    faltan = [c for c in ['inicio_ts', 'clima', 'duracion_s'] if c not in df.columns]
    if faltan:
        raise ValueError(f"Faltan columnas: {faltan}. Requiere: inicio_ts, clima, duracion_s.")

    # Cast de tipos robusto
    df['inicio_ts']  = pd.to_datetime(df['inicio_ts'], errors='coerce')
    df['duracion_s'] = pd.to_numeric(df['duracion_s'], errors='coerce')

    # Normaliza 'clima' a dos niveles canónicos
    mapa = {
        'bad': 'lluvioso', 'lluvioso': 'lluvioso', 'lluvia': 'lluvioso',
        'rain': 'lluvioso', 'rainy': 'lluvioso',
        'good': 'sin lluvia', 'clear': 'sin lluvia', 'sin lluvia': 'sin lluvia',
    }
    df['clima'] = (df['clima'].astype('string')
                             .str.strip().str.lower()
                             .map(mapa)
                             .fillna('sin lluvia'))

    return df


df = ren(df)


# --- Filtro de plausibilidad y (opcional) atípicos por IQR por grupo ---
def out_iqr(s: pd.Series):
    """
    Calcula máscara de atípicos por regla IQR (1.5*IQR) sobre una serie numérica.

    Parámetros
    ----------
    s : pd.Series
        Serie numérica (idealmente sin nulos).

    Retorna
    -------
    mask_out : pd.Series[bool]
        True en las posiciones consideradas atípicas (outliers).
    bounds : tuple[float, float]
        Límites (lo, hi) usados para clasificar outliers.
    """
    # Percentiles robustos al orden; asume s ya sin NaN fuera de esta función.
    q1, q3 = np.percentile(s, [25, 75])
    iqr = q3 - q1
    lo, hi = q1 - 1.5 * iqr, q3 + 1.5 * iqr
    mask_out = (s < lo) | (s > hi)
    return mask_out, (lo, hi)


# Filtro de plausibilidad duro por duración (MIN_S..MAX_S)
sab = df[(df['duracion_s'] >= MIN_S) & (df['duracion_s'] <= MAX_S)].copy()

# Remoción opcional de atípicos por grupo de clima
if RM_OUT:
    partes = []
    for lv in ['lluvioso', 'sin lluvia']:
        g = sab.loc[sab['clima'].eq(lv), 'duracion_s'].dropna()
        if g.empty:
            continue
        mk, _ = out_iqr(g)
        # Conserva solo valores no atípicos y reconstituye la etiqueta de clima
        partes.append(pd.DataFrame({'duracion_s': g[~mk], 'clima': lv}))
    sab = pd.concat(partes, ignore_index=True) if partes else sab

# Resultado filtrado para etapas posteriores
df_filtrado = sab.copy()
print("Dimensiones df_filtrado:", df_filtrado.shape)


## 3. Limpieza, validación y reporte reproducible

### **Objetivo**  
Aplicar reglas mínimas de calidad y plausibilidad para obtener un dataset consistente y trazable, listo para el EDA y las pruebas de hipótesis.

### **Qué hace este bloque y por qué**  
- **Estandariza esquema y tipos**: usa `ren(...)` para unificar columnas (`inicio_ts`, `clima`, `duracion_s`) y castear a `datetime`/numérico, evitando errores posteriores.  
- **Control de nulos y categorías**: descarta filas con nulos en variables clave y restringe `clima` a `{'lluvioso', 'sin lluvia'}` para garantizar comparabilidad.  
- **Plausibilidad de duración**: elimina duraciones no válidas (`<= 0 s`) y aplica límites realistas (`MIN_S`, `MAX_S`) para concentrarse en observaciones representativas.  
- **Contexto del caso**: se fuerza `dia = 'Sábado'` dado que el análisis se centra en sábados (si el caso cambia, esta asignación debe revisarse).  
- **Reporte reproducible**: imprime conteos de nulos, filas eliminadas por cada regla y tamaño final del dataset, facilitando auditoría y trazabilidad.

### **Supuestos y alcance**  
- Tras `ren(...)`, existen las columnas `inicio_ts`, `clima`, `duracion_s` y sus tipos son correctos.  
- Los umbrales `MIN_S`/`MAX_S` reflejan conocimiento del dominio (20–90 min para un trayecto Loop → O'Hare).  
- La codificación de `clima` ya está normalizada a dos niveles canónicos.

### **Resultados esperados**  
- `df_filtrado` listo para EDA y pruebas, con registros válidos y consistentes.  
- Un reporte claro de cuántas filas se removieron por nulos, duraciones no válidas y límites realistas.  

In [ ]:
# --- Limpieza con reglas realistas y reporte reproducible ---
def limpiar_y_validar_datos(df: pd.DataFrame,
                            MIN_S: int | None = None,
                            MAX_S: int | None = None,
                            verbose: bool = True) -> pd.DataFrame:
    """
    Limpia y valida el dataset aplicando reglas mínimas de calidad y plausibilidad.

    Pasos:
    1) Estandariza esquema y tipos con `ren(...)` (inicio_ts, clima, duracion_s).
    2) Reporta nulos iniciales por columna.
    3) Elimina filas con nulos en variables clave y restringe `clima` a
       {'lluvioso', 'sin lluvia'} (codificación canónica).
    4) Descarta duraciones no válidas (<= 0 s).
    5) Aplica filtros realistas opcionales de duración: MIN_S (inferior) y MAX_S (superior).
    6) Marca todos los registros como 'Sábado' (variable `dia` forzada para el caso de estudio).
    7) Emite un reporte reproducible (si `verbose=True`) con conteos de registros removidos.

    Parámetros
    ----------
    df : pd.DataFrame
        Datos de entrada.
    MIN_S : int | None
        Umbral inferior de duración en segundos. Si None, no se filtra por mínimo.
    MAX_S : int | None
        Umbral superior de duración en segundos. Si None, no se filtra por máximo.
    verbose : bool
        Si True, imprime un resumen de validación y limpieza.

    Retorna
    -------
    pd.DataFrame
        DataFrame limpio y validado, listo para EDA/pruebas.
    """
    # 1) Estandarización de esquema y tipos
    df_clean = ren(df.copy())
    n_total = len(df_clean)

    # 2) Métricas de calidad iniciales (nulos por variable clave)
    stats_iniciales = {
        'nulos_ts': df_clean['inicio_ts'].isna().sum(),
        'nulos_clima': df_clean['clima'].isna().sum(),
        'nulos_duracion': df_clean['duracion_s'].isna().sum()
    }

    # 3) Eliminación de nulos y normalización de categorías válidas en 'clima'
    df_clean = df_clean.dropna(subset=['inicio_ts', 'clima', 'duracion_s'])
    df_clean = df_clean[df_clean['clima'].isin(['lluvioso', 'sin lluvia'])]

    # 4) Duraciones no válidas (<= 0 s)
    mask_negativos = (df_clean['duracion_s'] <= 0) | (df_clean['duracion_s'].isna())
    n_negativos = mask_negativos.sum()
    df_clean = df_clean[~mask_negativos]

    # 5) Filtros realistas de duración (opcional)
    n_muy_cortos = 0
    n_muy_largos = 0
    if MIN_S is not None:
        mask_muy_cortos = df_clean['duracion_s'] < MIN_S
        n_muy_cortos = mask_muy_cortos.sum()
        df_clean = df_clean[~mask_muy_cortos]
    if MAX_S is not None:
        mask_muy_largos = df_clean['duracion_s'] > MAX_S
        n_muy_largos = mask_muy_largos.sum()
        df_clean = df_clean[~mask_muy_largos]

    # 6) Contexto del caso: todos los registros son sábados (se fuerza 'dia')
    #    Nota: si en otro dataset no aplica, quitar o adaptar esta línea.
    df_clean['dia'] = 'Sábado'

    # Reindex para un índice limpio tras filtrado
    df_clean = df_clean.reset_index(drop=True)

    # 7) Reporte reproducible
    if verbose:
        print("=== VALIDACIÓN Y LIMPIEZA DE DATOS ===")
        print('\n1. CALIDAD DE DATOS INICIAL:')
        print("-"*40)
        print(f"Filas totales iniciales: {n_total:,}")
        print("Valores nulos iniciales:")
        print(f"  • inicio_ts: {stats_iniciales['nulos_ts']}")
        print(f"  • clima: {stats_iniciales['nulos_clima']}")
        print(f"  • duracion_s: {stats_iniciales['nulos_duracion']}")

        print('\n2. ELIMINACIÓN DE VALORES ATÍPICOS / INVÁLIDOS:')
        print("-"*40)
        print(f"• Duración <= 0 s: {n_negativos}")
        if MIN_S is not None:
            print(f"• Duración < {MIN_S} s: {n_muy_cortos}")
        if MAX_S is not None:
            print(f"• Duración > {MAX_S} s: {n_muy_largos}")

        print('\n3. RESULTADO FINAL:')
        print("-"*40)
        print(f"Filas después de limpieza: {len(df_clean):,}")
        print(f"Filas correspondientes a sábados (forzado): {len(df_clean):,}")

        print('\n4. INFORMACIÓN DEL DATASET LIMPIO:')
        print("-"*40)
        print(f"Dimensiones: {df_clean.shape}")
        print(f"Rango de índice: {df_clean.index.min()} - {df_clean.index.max()}")

    return df_clean


# Ejecución de la limpieza con los umbrales globales
df_limpio = limpiar_y_validar_datos(df, MIN_S=MIN_S, MAX_S=MAX_S, verbose=True)
df_limpio.head()

## 4. AED orientado a decisiones

### **Objetivo**  
Caracterizar, en sábados, la intensidad temporal y la distribución de la duración de viajes por condición climática, para **elegir la prueba estadística adecuada** (paramétrica vs no paramétrica / bootstrap).

### **Qué hace este bloque y por qué**  
- **Validación de esquema/tipos**: garantiza presencia de `{'dia','inicio_ts','duracion_s','clima'}` y castea a tipos correctos.  
- **Filtrado de sábados**: usa `dia` normalizado; si no es confiable, infiere por `weekday` del timestamp.  
- **Normalización de clima**: consolida a `{'lluvioso','sin lluvia'}` (`clima_std`) para comparabilidad.  
- **Métricas clave**:  
  - Intensidad: viajes por **fecha** y por **hora** (picos/vales).  
  - Duración: media, mediana, rango, DE y **CV**.  
  - Por clima: conteo, porcentaje y media de duración.  
  - Forma: **sesgo** y **curtosis** para detectar asimetrías/colas.  
- **Visualizaciones (2×3)**:  
  a) Serie temporal por clima (conteo diario).  
  b) Viajes por hora (barras).  
  c) Viajes por clima (barras).  
  d) Duración promedio por hora (línea).  
  e) Histogramas por clima.  
  f) Boxplot por clima.

### **Criterios para elegir la prueba**  
- Distribuciones similares y sin colas extremas --> **t de Student** (Welch si varianzas desiguales).  
- Asimetría marcada, colas pesadas o tamaños muy distintos --> **Mann–Whitney** o **bootstrap** (diferencia de medias/medianas).

### **Supuestos y alcance**  
- `df_clean` ya fue sometido a limpieza/validación previa.  
- La asignación a sábados es coherente con el objetivo del estudio.

### **Resultados esperados**  
- Panorama claro de intensidad temporal y forma de la distribución por clima.  
- Evidencia visual/numérica para sustentar la elección de la prueba estadística en la siguiente sección.  

In [ ]:
# --- Exploratorio orientado a decisiones — guía la prueba a usar ---
def aed(df_clean: pd.DataFrame, GRAF: bool = True) -> None:
    """
    Realiza un EDA compacto, enfocado en decidir la prueba estadística adecuada
    para comparar la duración de viajes en sábado según la condición climática.

    Flujo:
    1) Validación de esquema mínimo y tipado robusto.
    2) Normalización de 'dia' para asegurar el subconjunto de sábados.
       - Si 'dia' no es confiable, se usa el día de la semana de 'inicio_ts'.
    3) Normalización de clima a {'lluvioso', 'sin lluvia'} en 'clima_std'.
    4) Métricas clave:
       - Intensidad temporal (viajes por fecha/hora).
       - Estadísticos de duración (media, mediana, rango, DE, CV).
       - Resumen por clima (n, %, media).
       - Forma de la distribución (sesgo, curtosis).
    5) Visualizaciones (2×3) si GRAF=True:
       a) Serie temporal: conteo diario por clima.
       b) Viajes por hora.
       c) Viajes por clima.
       d) Duración promedio por hora.
       e) Histogramas de duración por clima.
       f) Boxplot de duración por clima.

    Decisión estadística (orientativa):
    - Si las distribuciones por clima son similares y sin colas extremas → t de Student
      (con Welch si varianzas desiguales).
    - Si hay asimetría/colas o tamaños muy distintos → prueba no paramétrica
      (Mann–Whitney) o bootstrap de diferencia de medias/medianas.

    Parámetros
    ----------
    df_clean : pd.DataFrame
        Dataset ya limpiado/validado con al menos las columnas:
        {'dia','inicio_ts','duracion_s','clima'}.
    GRAF : bool
        Si True, genera figura con 6 subplots; si False, solo imprime métricas.

    Retorna
    -------
    None
        Imprime resultados y, si procede, dibuja figuras.
    """
    # Chequeos y normalizaciones básicas
    req = {'dia', 'inicio_ts', 'duracion_s', 'clima'}
    missing = req - set(df_clean.columns)
    if missing:
        raise KeyError(f"Faltan columnas requeridas: {sorted(missing)}")

    df = df_clean.copy()

    # Asegurar datetime y numérico; descartar nulos críticos
    df['inicio_ts']  = pd.to_datetime(df['inicio_ts'], errors='coerce')
    df['duracion_s'] = pd.to_numeric(df['duracion_s'], errors='coerce')
    df = df.dropna(subset=['inicio_ts', 'duracion_s'])

    # Normalizar 'dia' (ej. 'Sábado'/'sabado'/'Saturday' -> 'sabado')
    dia_norm = (df['dia'].astype(str)
                .str.normalize('NFKD').str.encode('ascii', 'ignore').str.decode('utf-8')
                .str.lower().str.strip())
    df['dia_norm'] = dia_norm

    # Subconjunto de sábados; si no hay, inferir por weekday (5 = sábado)
    sab = df[df['dia_norm'].eq('sabado')].copy()
    if sab.empty:
        sab = df[df['inicio_ts'].dt.dayofweek.eq(5)].copy()
        if sab.empty:
            raise RuntimeError("No hay registros de sábado. Revisa 'dia' o el dtype de 'inicio_ts'.")

    # Normalizar clima a {'lluvioso','sin lluvia'} conservando valores originales si no mapean
    clima_map = {
        'lluvioso': 'lluvioso', 'con lluvia': 'lluvioso', 'rainy': 'lluvioso',
        'sin lluvia': 'sin lluvia', 'no lluvioso': 'sin lluvia', 'not rainy': 'sin lluvia', 'seco': 'sin lluvia'
    }
    sab['clima_std'] = (sab['clima'].astype(str).str.lower().str.strip().map(clima_map)
                        .fillna(sab['clima'].astype(str)))

    # --- Métricas impresas (EDA numérico) ---
    print("="*60)
    print("ANÁLISIS EXPLORATORIO DE DATOS: Viajes en Sábado (Loop → O'Hare)")
    print("="*60)

    # 1) Análisis temporal
    print("\n1. ANÁLISIS TEMPORAL")
    print("-"*40)
    viajes_por_fecha = sab.groupby(sab['inicio_ts'].dt.date).size()
    print(f"• Viajes por día: {viajes_por_fecha.mean():.1f} ± {viajes_por_fecha.std():.1f}")
    print(f"• Rango: {viajes_por_fecha.min()} - {viajes_por_fecha.max()} viajes/día")
    viajes_por_hora = sab.groupby(sab['inicio_ts'].dt.hour).size()
    print(f"• Hora pico: {viajes_por_hora.idxmax()}:00 hrs ({viajes_por_hora.max()} viajes)")
    print(f"• Horas valle: {viajes_por_hora.idxmin()}:00 hrs ({viajes_por_hora.min()} viajes)")

    # 2) Estadísticos de duración
    print("\n2. ANÁLISIS DE DURACIÓN")
    print("-"*40)
    dsc = sab['duracion_s'].describe()
    mean = dsc['mean']; std = dsc['std']
    print(f"• Promedio: {mean/60:.1f} min | Mediana: {dsc['50%']/60:.1f} min")
    print(f"• Rango: {dsc['min']/60:.1f} - {dsc['max']/60:.1f} min | DE: {std/60:.1f} min")
    cv = (std/mean)*100 if mean != 0 else np.nan
    print(f"• CV: {cv:.1f}%")

    # 3) Resumen por clima
    print("\n3. ANÁLISIS POR CONDICIÓN CLIMÁTICA")
    print("-"*40)
    clima_counts = sab['clima_std'].value_counts()
    clima_duration = sab.groupby('clima_std')['duracion_s'].agg(['mean', 'std', 'count'])
    for c in clima_counts.index:
        count = clima_counts[c]
        percent = (count / len(sab)) * 100
        mean_dur = clima_duration.loc[c, 'mean'] / 60
        print(f"• {c.title()}: {count} viajes ({percent:.1f}%), {mean_dur:.1f} min promedio")

    # 4) Forma de la distribución
    print("\n4. ESTADÍSTICAS DE DISTRIBUCIÓN")
    print("-"*40)
    print(f"• Sesgo: {sab['duracion_s'].skew():.3f} | Kurtosis: {sab['duracion_s'].kurt():.3f}")

    # --- Visualizaciones ---
    if not GRAF:
        return

    fig, axes = plt.subplots(2, 3, figsize=(18, 12))

    # a) Serie temporal: días lluviosos vs secos por fecha (sin crear columnas nuevas)
    sab['inicio_ts'] = pd.to_datetime(sab['inicio_ts'], errors='coerce')
    sab_ok = sab.dropna(subset=['inicio_ts'])
    freq = (pd.crosstab(
                sab_ok['inicio_ts'].dt.normalize(),   # fecha (00:00)
                sab_ok['clima_std']                    # categorías normalizadas
            )
            .reindex(columns=['sin lluvia', 'lluvioso'], fill_value=0)
            .sort_index())

    axes[0,0].plot(freq.index, freq['sin lluvia'].to_numpy(), marker='o', linewidth=2, label='Sin lluvia')
    axes[0,0].plot(freq.index, freq['lluvioso'].to_numpy(),   marker='o', linewidth=2, label='Lluvioso')
    axes[0,0].set_title('Días Lluviosos vs Secos por Fecha')
    axes[0,0].set_xlabel('Fecha'); axes[0,0].set_ylabel('Frecuencia')
    axes[0,0].tick_params(axis='x', rotation=45)
    axes[0,0].legend(title='Clima')

    # b) Viajes por hora (barras)
    axes[0,1].bar(viajes_por_hora.index, viajes_por_hora.values, alpha=0.7)
    axes[0,1].set_title("Viajes por hora (Sábados)")
    axes[0,1].set_xlabel("Hora"); axes[0,1].set_ylabel("Cantidad de viajes")
    axes[0,1].set_xticks(range(0, 24, 2))
    axes[0,1].grid(True, axis='y', lw=0.5, alpha=0.6)

    # c) Viajes por clima (barras)
    viajes_por_clima = sab['clima_std'].value_counts()
    axes[0,2].bar(viajes_por_clima.index, viajes_por_clima.values, alpha=0.7)
    axes[0,2].set_title("Viajes por condición climática (Sábados)")
    axes[0,2].set_ylabel("Cantidad de viajes")

    # d) Duración promedio por hora (línea)
    duracion_por_hora = sab.groupby(sab['inicio_ts'].dt.hour)['duracion_s'].mean()
    axes[1,0].plot(duracion_por_hora.index, duracion_por_hora.values, marker='o', linewidth=2)
    axes[1,0].set_title("Duración promedio por hora (Sábados)")
    axes[1,0].set_xlabel("Hora"); axes[1,0].set_ylabel("Duración promedio (s)")
    axes[1,0].set_xticks(range(0, 24, 2))
    axes[1,0].grid(True, lw=0.5, alpha=0.6)

    # e) Histogramas por clima
    a = sab.loc[sab['clima_std'].eq('sin lluvia'), 'duracion_s'].astype(float).values
    b = sab.loc[sab['clima_std'].eq('lluvioso'),    'duracion_s'].astype(float).values
    axes[1,1].hist(a, bins=30, alpha=0.6, label="Sin lluvia")
    axes[1,1].hist(b, bins=30, alpha=0.6, label="Lluvioso")
    axes[1,1].set_title("Sábado: histograma de duración por clima")
    axes[1,1].set_xlabel("Duración (s)"); axes[1,1].set_ylabel("Frecuencia")
    axes[1,1].legend(); axes[1,1].grid(True, lw=0.5, alpha=0.6)

    # f) Boxplot por clima (completa 2×3)
    sab.boxplot(column='duracion_s', by='clima_std', ax=axes[1,2])
    axes[1,2].set_title("Duración por clima (boxplot)")
    axes[1,2].set_xlabel(""); axes[1,2].set_ylabel("Duración (s)")
    fig.suptitle("")

    plt.tight_layout()
    plt.show()


# Ejemplo de uso (si tu DataFrame limpio se llama df_limpio):
aed(df_limpio, GRAF=GRAF)

## 5. Utilidades: normalidad, tamaños de efecto e IC bootstrap

### **Objetivo**  
Proveer funciones reutilizables para: (i) evaluar **normalidad** de las muestras, (ii) calcular **tamaños de efecto** comparables (g de Hedges) y (iii) estimar **intervalos de confianza bootstrap** (medias y medianas) para diferencias entre grupos.

### **Qué hace este bloque y por qué**  
- **`pnorm`**: aplica una prueba de normalidad adaptativa al tamaño muestral (Shapiro–Wilk hasta n=500; D’Agostino–Pearson en n grandes). Evita conclusiones débiles en n<8 devolviendo 1.0.  
- **`g_hedges`**: estima el tamaño de efecto para dos grupos independientes corrigiendo el sesgo de `d` en muestras finitas; devuelve `NaN` si no hay grados de libertad y 0 si la DE agrupada es 0.  
- **`ci_boot_means` / `ci_boot_medians`**: generan IC del 95% por percentiles para la diferencia de medias/medianas usando remuestreo con reemplazo y semilla reproducible (`RNG_SEED`).  

### **Supuestos y alcance**  
- Las entradas (`x`, `y`) son arrays numéricos independientes por grupo y sin NaN.  
- Los IC bootstrap reflejan la variabilidad muestral bajo remuestreo con reemplazo; no asumen normalidad.  
- Para tamaños de muestra muy pequeños (n<8), los tests de normalidad son poco informativos.

### **Resultados esperados**  
- p-valores de normalidad coherentes con el tamaño muestral.  
- Tamaños de efecto **g** interpretables (pequeño ~0.2, mediano ~0.5, grande ~0.8, guía orientativa).  
- IC 95% (medias/medianas) listos para reporte y para acompañar la prueba de hipótesis principal.


In [ ]:
# --- Utilidades de normalidad y tamaños de efecto ---
def pnorm(a: np.ndarray) -> float:
    """
    Devuelve el p-valor de una prueba de normalidad adecuada al tamaño muestral.
    
    Reglas:
    - n < 8  → devuelve 1.0 (no hay potencia para testear normalidad).
    - 8 ≤ n ≤ 500  → Shapiro–Wilk (potente en muestras pequeñas/medianas).
    - n > 500 → D’Agostino–Pearson (stats.normaltest), más estable en n grandes.
    """
    a = np.asarray(a, float)
    n = a.size
    if n < 8:
        return 1.0
    return stats.shapiro(a).pvalue if n <= 500 else stats.normaltest(a).pvalue


def g_hedges(x: np.ndarray, y: np.ndarray) -> float:
    """
    Calcula el tamaño de efecto g de Hedges para dos grupos independientes.

    g corrige el sesgo de Cohen's d en muestras finitas:
        g = J * d,  con  J = 1 - 3/(4*(n1+n2) - 9)

    Retorna NaN si algún grupo tiene n < 2; retorna 0 si la DE agrupada es 0.
    """
    x, y = np.asarray(x, float), np.asarray(y, float)
    n1, n2 = len(x), len(y)
    if n1 < 2 or n2 < 2:
        return np.nan

    s1, s2 = x.var(ddof=1), y.var(ddof=1)
    df_pooled = (n1 + n2 - 2)
    if df_pooled <= 0:
        return np.nan

    # DE agrupada (pooled)
    sp = np.sqrt(((n1 - 1) * s1 + (n2 - 1) * s2) / df_pooled)
    if not np.isfinite(sp) or sp == 0:
        return 0.0

    d = (x.mean() - y.mean()) / sp
    J = 1 - 3 / (4 * (n1 + n2) - 9) if (n1 + n2) > 2 else 1.0
    return J * d


def ci_boot_means(x: np.ndarray, y: np.ndarray, B: int = NBOOT, seed: int = RNG_SEED) -> tuple[float, float]:
    """
    IC bootstrap (percentil) del 95% para la diferencia de medias (x - y).

    Parámetros
    ----------
    x, y : arrays numéricos
    B : int
        Número de réplicas bootstrap.
    seed : int
        Semilla para reproducibilidad.

    Retorna
    -------
    (lo, hi) : tupla de floats
        Límites inferior y superior del IC al 95% por percentiles (2.5, 97.5).
    """
    rng = np.random.default_rng(seed)
    x, y = np.asarray(x, float), np.asarray(y, float)
    n1, n2 = len(x), len(y)
    dif = np.empty(B)

    for b in range(B):
        dif[b] = rng.choice(x, n1, True).mean() - rng.choice(y, n2, True).mean()

    lo, hi = np.percentile(dif, [2.5, 97.5])
    return lo, hi


def ci_boot_medians(x: np.ndarray, y: np.ndarray, B: int = NBOOT, seed: int = RNG_SEED) -> tuple[float, float]:
    """
    IC bootstrap (percentil) del 95% para la diferencia de medianas (x - y).
    """
    rng = np.random.default_rng(seed)  # semilla una sola vez
    x, y = np.asarray(x, float), np.asarray(y, float)
    n1, n2 = len(x), len(y)
    dif = np.empty(B)

    for b in range(B):
        xb = rng.choice(x, size=n1, replace=True)
        yb = rng.choice(y, size=n2, replace=True)
        dif[b] = np.median(xb) - np.median(yb)

    lo, hi = np.percentile(dif, [2.5, 97.5])
    return lo, hi

## 6. Supuestos, prueba de dos grupos y tamaños de efecto

### **Objetivo**  
Contrastar la **duración** de viajes en sábado entre **lluvioso** y **sin lluvia**, verificando supuestos (normalidad y varianzas) y reportando evidencia completa: prueba seleccionada, p-valor, tamaño de efecto e IC bootstrap.

### **Qué hace este bloque y por qué**  
- **Limpieza opcional de outliers** por IQR dentro de cada clima para no sesgar la comparación.  
- **Normalidad adaptativa** (`pnorm`) y **Levene** (centro=mediana) para decidir entre:  
  - **t de Student** (Welch si varianzas desiguales) cuando ambas muestras parecen normales.  
  - **Mann–Whitney U** cuando no se cumple normalidad.  
- **Intervalos de confianza bootstrap (95%)** para la diferencia (medias o medianas), y **tamaño de efecto**:  
  - `g` de Hedges en el caso paramétrico.  
  - `r` derivado de U→z en el caso no paramétrico.  
- **Salida estructurada** con diferencia puntual en minutos, IC, p-valor, estadístico y N por grupo.

### **Supuestos y alcance**  
- `df_limpio` ya contiene solo sábados y `clima ∈ {lluvioso, sin lluvia}`.  
- Bootstrap asume remuestreo independiente por grupo.  
- La interpretación de tamaños de efecto sigue guías orientativas (p.ej., g≈0.2 pequeño, 0.5 mediano, 0.8 grande).

### **Resultados esperados**  
- Decisión clara sobre la prueba empleada (t/Welch vs Mann–Whitney).  
- Evidencia cuantitativa completa: **diferencia en minutos**, **IC 95%**, **p-valor** y **tamaño de efecto** lista para el informe final.


In [ ]:
# --- Prueba de hipótesis con árbol de decisión + tamaños de efecto ---
def verificar_supuestos_y_prueba(df_clean: pd.DataFrame,
                                 alpha: float = ALFA,
                                 rm_outliers: bool = RM_OUT) -> dict:
    """
    Verifica supuestos (normalidad/varianzas) y ejecuta una prueba de dos grupos
    para comparar duración en sábado por clima: 'lluvioso' vs 'sin lluvia'.

    Árbol de decisión:
    - Si ambas muestras pasan normalidad (p>α) → t-test (Welch si varianzas desiguales).
    - En otro caso → Mann–Whitney U (no paramétrica).
    Además, reporta tamaños de efecto (g de Hedges o r) e IC bootstrap (medias o medianas).

    Retorna un diccionario con la prueba usada, p-valor, estadístico,
    diferencia puntual en minutos, IC al 95% en minutos, tamaños muestrales y efecto.
    """
    sab = df_clean.copy()  # todos son 'Sábado' post-limpieza
    if sab.empty:
        raise RuntimeError("Dataset vacío tras la limpieza.")

    # (Opcional) quitar atípicos por IQR dentro de cada nivel de clima
    if rm_outliers:
        sab_f = []
        for lv in ['lluvioso', 'sin lluvia']:
            g = sab[sab['clima'].eq(lv)].copy()
            if g.empty:
                continue
            mk, _ = out_iqr(g['duracion_s'])
            sab_f.append(g.loc[~mk])
        sab = pd.concat(sab_f, ignore_index=True) if sab_f else sab

    # Definir grupos (en segundos)
    grupo_lluvioso   = sab.loc[sab['clima'].eq('lluvioso'),   'duracion_s'].astype(float).values
    grupo_sin_lluvia = sab.loc[sab['clima'].eq('sin lluvia'), 'duracion_s'].astype(float).values
    n_ll, n_sl = len(grupo_lluvioso), len(grupo_sin_lluvia)
    if n_ll < 2 or n_sl < 2:
        raise RuntimeError("Tamaño de muestra insuficiente para realizar pruebas estadísticas.")

    print("="*38)
    print("VERIFICACIÓN DE SUPUESTOS ESTADÍSTICOS")
    print("="*38)

    # 1) Normalidad
    print("\n1. PRUEBAS DE NORMALIDAD:")
    print("-"*27)
    p_norm_ll = pnorm(grupo_lluvioso)
    p_norm_sl = pnorm(grupo_sin_lluvia)
    print(f"  Lluvioso    → p={p_norm_ll:.4g}")
    print(f"  Sin lluvia  → p={p_norm_sl:.4g}")

    # 2) Homogeneidad de varianzas (Levene con centro en la mediana = robusto)
    p_var = stats.levene(grupo_lluvioso, grupo_sin_lluvia, center='median').pvalue
    print("\n2. IGUALDAD DE VARIANZAS (Levene, centro=mediana):")
    print("-"*50)
    print(f"  p={p_var:.4g}")

    # 3) Selección de prueba
    use_mean = (p_norm_ll > alpha) and (p_norm_sl > alpha)
    if use_mean:
        print("\n3. PRUEBA: medias (t de Student/Welch)")
        equal_var = p_var > alpha
        stat, p_valor = stats.ttest_ind(grupo_lluvioso, grupo_sin_lluvia, equal_var=equal_var)
        tam = g_hedges(grupo_lluvioso, grupo_sin_lluvia)  # tamaño de efecto
        dif_obs = grupo_lluvioso.mean() - grupo_sin_lluvia.mean()
        lo, hi  = ci_boot_means(grupo_lluvioso, grupo_sin_lluvia, B=NBOOT, seed=RNG_SEED)
        stat_name = "media"
        interpret = ("pequeño" if abs(tam) < 0.2 else
                     "pequeño–moderado" if abs(tam) < 0.5 else
                     "moderado" if abs(tam) < 0.8 else "grande")
        punto_est = dif_obs / 60
        lo_m, hi_m = lo / 60, hi / 60
    else:
        print("\n3. PRUEBA: medianas (Mann–Whitney U)")
        stat, p_valor = stats.mannwhitneyu(grupo_lluvioso, grupo_sin_lluvia, alternative='two-sided')
        dif_obs = np.median(grupo_lluvioso) - np.median(grupo_sin_lluvia)
        lo, hi  = ci_boot_medians(grupo_lluvioso, grupo_sin_lluvia, B=NBOOT, seed=RNG_SEED)
        # Tamaño de efecto r desde U → z estandarizado
        n_ll, n_sl = len(grupo_lluvioso), len(grupo_sin_lluvia)
        mu = n_ll * n_sl / 2
        sigma = np.sqrt(n_ll * n_sl * (n_ll + n_sl + 1) / 12)
        z = (stat - mu) / sigma
        tam = abs(z) / np.sqrt(n_ll + n_sl)
        stat_name = "mediana"
        interpret = None
        punto_est = dif_obs / 60
        lo_m, hi_m = lo / 60, hi / 60

    # 4) Reporte de resultados
    print("\n4. RESULTADOS:")
    print("-"*40)
    print(f"• Diferencia de {stat_name}s (min): {punto_est:+.2f}")
    print(f"• IC {int((1-ALFA)*100)}% (min): [{lo_m:.2f}, {hi_m:.2f}]")
    print(f"• Estadístico: {stat:.3f} | p-valor: {p_valor:.4g}")
    if use_mean:
        print(f"• Tamaño de efecto (g de Hedges): {tam:.3f} ({interpret})")
    else:
        print(f"• Tamaño de efecto (r de Mann-Whitney): {tam:.3f}")

    concl = ("Diferencia estadísticamente significativa"
             if p_valor < alpha else
             "No se encontró diferencia estadísticamente significativa")
    print(f"• Conclusión: {concl} (α={alpha})")

    # Salida estructurada para uso posterior (tablas/figuras/reporte)
    return {
        'prueba': 't (Student/Welch)' if use_mean else 'Mann–Whitney U',
        'p_valor': float(p_valor),
        'estadistico': float(stat),
        'use_mean': use_mean,
        'dif_minutos': float(punto_est),
        'ic_minutos': (float(lo_m), float(hi_m)),
        'n_lluvioso': int(len(grupo_lluvioso)),
        'n_sin_lluvia': int(len(grupo_sin_lluvia)),
        'tam_efecto': float(tam),
        'tipo_efecto': 'g_hedges' if use_mean else 'r_mannwhitney'
    }


# Ejecutar (usa los flags globales definidos previamente)
resultados_prueba = verificar_supuestos_y_prueba(df_limpio, alpha=ALFA, rm_outliers=RM_OUT)
resultados_prueba

## 7. Comparación visual (violín + box + jitter) con énfasis en media/mediana

### **Objetivo**  
Ofrecer una comparación **intuitiva y robusta** de la distribución de la duración entre **lluvioso** y **sin lluvia**, resaltando el estadístico central (media o mediana) coherente con la prueba inferencial seleccionada.

### **Qué hace este bloque y por qué**  
- **Violín**: revela la **forma** de la distribución (asimetrías, multimodalidad, colas).  
- **Boxplot**: resume **mediana** e **IQR**, complementando la lectura del violín.  
- **Jitter**: muestra la **dispersión puntual** y sugiere el **tamaño muestral** (se añade además `n` bajo las categorías).  
- **IQR por grupo (opcional)**: si `rm_outliers=True`, elimina atípicos extremos dentro de cada clima (1.5·IQR), evitando que pocos valores extremos dominen la percepción.  
- **Coherencia inferencial**: el marcador central resalta **mediana** (Mann–Whitney) o **media** (t/Welch), armonizando visual y prueba.

### **Supuestos y alcance**  
- `df_limpio` contiene `duracion_s` (segundos) y `clima ∈ {lluvioso, sin lluvia}`.  
- El filtrado por IQR aquí es **visual/diagnóstico** y se declara por separado en la sección inferencial.  
- Si un grupo queda sin datos (p. ej., tras filtros), la función evita fallos y avisa.

### **Resultados esperados**  
- Una lectura visual clara de **posición**, **dispersión** y **forma** por clima, con el estadístico central resaltado según la prueba.  
- Contexto de **tamaño muestral** por grupo (etiquetas `n=`), útil para interpretar densidad vs. variabilidad.


In [ ]:
def comparacion_visual_violin_box_jitter(
    df_limpio: pd.DataFrame,
    rm_outliers: bool = True,
    seed: int = RNG_SEED,
    use_mean: bool | None = None,
    resultados: dict | None = None,
    color_ll: str = "blue",
    color_sl: str = "orange",
) -> None:
    """
    Visualización robusta de 'duracion_s' por clima ('lluvioso' vs 'sin lluvia'):
      • Violín: forma/densidad de la distribución.
      • Boxplot: mediana e IQR (sin outliers si ya filtramos por IQR).
      • Jitter: dispersión puntual (muestra N y variabilidad).
      • Marcador central: resalta MEDIA o MEDIANA según `use_mean` o `resultados`.

    Parámetros
    ----------
    df_limpio : pd.DataFrame
        Dataset limpio (solo sábados) con columnas 'duracion_s' (numérico) y 'clima'.
    rm_outliers : bool
        Si True, remueve atípicos por la regla 1.5·IQR **por grupo de clima**.
    seed : int
        Semilla para reproducibilidad del jitter (ruido horizontal).
    use_mean : bool | None
        Si True, resalta **media**; si False, **mediana**. Si None, toma de `resultados['use_mean']`
        (por ejemplo, coherente con t-test vs Mann–Whitney); por defecto False (mediana).
    resultados : dict | None
        Diccionario opcional (salida de `verificar_supuestos_y_prueba`) para inferir `use_mean`.
    color_ll : str
        Color para el grupo 'lluvioso'.
    color_sl : str
        Color para el grupo 'sin lluvia'.

    Retorna
    -------
    None
        Dibuja la figura en pantalla.
    """
    # 1) Elegir estadístico central a resaltar
    if use_mean is None:
        use_mean = bool(resultados.get('use_mean', False)) if isinstance(resultados, dict) else False

    # 2) Preparación de datos y tipos
    sab = (
        df_limpio[["duracion_s", "clima"]]
        .copy()
        .assign(duracion_s=lambda d: pd.to_numeric(d["duracion_s"], errors="coerce"))
        .dropna(subset=["duracion_s", "clima"])
    )

    # 3) Filtrado IQR por grupo (opcional)
    if rm_outliers and not sab.empty:
        def _keep(s: pd.Series) -> pd.Series:
            q1, q3 = s.quantile([0.25, 0.75])
            iqr = q3 - q1
            lo, hi = q1 - 1.5 * iqr, q3 + 1.5 * iqr
            return s.between(lo, hi)

        sab = sab.loc[sab.groupby("clima")["duracion_s"].transform(_keep)]

    # 4) Extraer arrays por grupo (en segundos, consistente con análisis)
    vals_ll = sab.loc[sab["clima"].eq("lluvioso"), "duracion_s"].to_numpy(float)
    vals_sl = sab.loc[sab["clima"].eq("sin lluvia"), "duracion_s"].to_numpy(float)
    datos   = [vals_ll, vals_sl]
    ns      = [vals_ll.size, vals_sl.size]

    # 5) Manejo robusto si falta alguno de los grupos
    if ns[0] == 0 and ns[1] == 0:
        print("Sin datos para graficar después de filtros.")
        return

    # 6) Figura
    fig, ax = plt.subplots(figsize=(8, 5), constrained_layout=True)

    # Violines (sin extremos para no duplicar con boxplot)
    vp = ax.violinplot(datos, showmeans=False, showextrema=False)
    for i, body in enumerate(vp.get("bodies", []), start=1):
        body.set_facecolor(color_ll if i == 1 else color_sl)
        body.set_edgecolor("black")
        body.set_alpha(0.28)

    # Boxplots (sin mostrar outliers porque ya controlamos IQR)
    bp = ax.boxplot(
        datos,
        showfliers=False,
        patch_artist=True,
        medianprops=dict(color="red", linewidth=1.4),
    )
    for box in bp["boxes"]:
        box.set_facecolor("white")
        box.set_linewidth(1.3)

    # Jitter: puntos individuales (da sensación de N y dispersión)
    rng = np.random.default_rng(seed)
    for i, arr in enumerate(datos, start=1):
        if arr.size == 0:
            continue
        x = rng.normal(loc=i, scale=0.05, size=arr.size)
        ax.plot(x, arr, "o", ms=2.2, alpha=0.22, color=(color_ll if i == 1 else color_sl))

    # Marcador central (media o mediana)
    stat_vals = [(a.mean() if use_mean else np.median(a)) if a.size else np.nan for a in datos]
    ax.scatter(
        [1, 2], stat_vals,
        marker="o", s=30, c="red", edgecolors="red", linewidths=1.2, zorder=3,
        label=("Media" if use_mean else "Mediana"),
    )

    # Ejes y estilo
    ax.set_xticks([1, 2])
    ax.set_xticklabels(["Lluvioso", "Sin lluvia"])
    ax.set_title("Sábado: Duración por clima", fontsize=13, fontweight="bold")
    ax.set_xlabel("Clima")
    ax.set_ylabel("Duración (s)")
    ax.grid(True, lw=0.6, alpha=0.55, axis="y")
    ax.legend(frameon=False, loc="upper right")

    # Formato es-MX del eje Y si el formateador está disponible
    try:
        ax.yaxis.set_major_formatter(FuncFormatter(fmt_miles_es))
    except Exception:
        pass

    # 7) Contexto adicional en eje X: n por grupo (ayuda a leer densidad)
    for i, n in enumerate(ns, start=1):
        ax.text(i, ax.get_ylim()[0], f"n={n}", ha="center", va="bottom", fontsize=9)

    plt.show()


# Ejecución: por defecto (coherente con Mann–Whitney) resalta MEDIANA
if GRAF:
    comparacion_visual_violin_box_jitter(df_limpio, rm_outliers=True, seed=RNG_SEED)


## 8. Conclusiones

### **Objetivo**  
Entregar una **lectura clara y accionable** del contraste **lluvioso vs sin lluvia**, combinando significancia estadística, **magnitud práctica** (tamaño de efecto) y **robustez** (IC bootstrap), para apoyar decisiones operativas y de comunicación.

### **Qué aporta este bloque**  
- **Señal estadística**: prueba seleccionada (t/Welch o Mann–Whitney) con p-valor y verificación del IC.  
- **Magnitud práctica**: tamaño de efecto (`g` o `r`) con interpretación (trivial/pequeño/mediano/grande).  
- **Robustez**: IC bootstrap (réplicas `B` y `seed`) y nota sobre IQR por grupo si se aplicó.  
- **Direccionalidad**: diferencia expresada en **minutos** (lluvioso − sin lluvia) y, si se proporciona una referencia, su **impacto porcentual**.  
- **Acciones**: recomendaciones según **signo** y **magnitud** del efecto.

### **Resultado esperado**  
Un bloque Markdown que el lector no técnico puede interpretar rápidamente (¿hay diferencia?, ¿cuánto?, ¿qué implica?) y que también satisface criterios técnicos (IC, tamaño de efecto, supuestos), dejando claro **qué hacer después**.


In [ ]:
# --- Conclusiones (Markdown) ---
def conclusiones_markdown(
    r: dict,
    alpha: float = ALFA,
    contexto: dict | None = None,
) -> None:
    """
    Genera conclusiones ejecutivas en Markdown para la comparación 'lluvioso' vs 'sin lluvia'.

    Parámetros
    ----------
    r : dict
        Salida de `verificar_supuestos_y_prueba(...)` con claves:
        - 'prueba' : str
        - 'p_valor' : float
        - 'estadistico' : float
        - 'use_mean' : bool
        - 'dif_minutos' : float         # (lluvioso - sin lluvia) en minutos
        - 'ic_minutos' : (float, float) # IC 95% en minutos
        - 'n_lluvioso' : int
        - 'n_sin_lluvia' : int
        - 'tam_efecto' : float
        - 'tipo_efecto' : {'g_hedges', 'r_mannwhitney'}
    alpha : float
        Nivel de significancia para interpretar p-valor y IC.
    contexto : dict | None
        Información opcional para enriquecer la interpretación, p. ej.:
        - 'rm_outliers' : bool
        - 'B' : int (réplicas bootstrap)
        - 'seed' : int (semilla)
        - 'baseline_ref_minutes' : float (p. ej., media de 'sin lluvia' en min)
        - 'kpi' : str (nombre del KPI, por defecto 'duración (min)')
        - 'ruta' : str (contexto del trayecto, p. ej. "Loop → O'Hare")

    Retorna
    -------
    None
        Renderiza un bloque Markdown con conclusiones y recomendaciones.
    """
    ctx = contexto or {}
    kpi = ctx.get('kpi', 'duración (min)')
    ruta = ctx.get('ruta', "Sábado")
    B = ctx.get('B', NBOOT)
    seed = ctx.get('seed', RNG_SEED)
    rm_out = ctx.get('rm_outliers', None)
    base_ref = ctx.get('baseline_ref_minutes', None)

    # --- Lecturas base
    p = float(r['p_valor'])
    lo, hi = r['ic_minutos']
    dif = float(r['dif_minutos'])  # lluvioso - sin lluvia (min)
    n_ll, n_sl = int(r['n_lluvioso']), int(r['n_sin_lluvia'])
    test = r['prueba']
    eff = float(r['tam_efecto'])
    eff_kind = r['tipo_efecto']  # 'g_hedges' o 'r_mannwhitney'
    sig = p < alpha
    cruza_cero = (lo <= 0.0 <= hi)

    # --- Etiquetas de tamaño de efecto y dirección ---
    if eff_kind == 'g_hedges':
        # Guías orientativas para g
        if abs(eff) < 0.2: mag = "trivial"
        elif abs(eff) < 0.5: mag = "pequeño"
        elif abs(eff) < 0.8: mag = "moderado"
        else: mag = "grande"
        eff_txt = f"g de Hedges = {eff:.3f} ({mag})"
        guias_txt = "≈0.2 pequeño, 0.5 moderado, 0.8 grande"
    else:
        # Guías orientativas para r
        if abs(eff) < 0.1: mag = "trivial"
        elif abs(eff) < 0.3: mag = "pequeño"
        elif abs(eff) < 0.5: mag = "mediano"
        else: mag = "grande"
        eff_txt = f"r de Mann–Whitney = {eff:.3f} ({mag})"
        guias_txt = "≈0.1 pequeño, 0.3 mediano, 0.5 grande"

    dir_txt = "mayor" if dif > 0 else ("menor" if dif < 0 else "igual")
    sentido_txt = (
        f"En promedio/mediana, **lluvioso** es **{dir_txt}** que **sin lluvia**"
        if dif != 0 else
        "No se observa diferencia puntual entre **lluvioso** y **sin lluvia**"
    )

    # --- Impacto porcentual (opcional, requiere referencia) ---
    pct_txt = ""
    if base_ref and base_ref > 0:
        pct = (dif / base_ref) * 100.0
        pct_txt = f" (~{pct:+.1f}% vs. referencia sin lluvia ≈ {base_ref:.2f} min)"

    # --- Nota de robustez
    robustez_bits = []
    robustez_bits.append(f"IC bootstrap (B={B}, seed={seed})")
    if rm_out is not None:
        robustez_bits.append("IQR por grupo activo" if rm_out else "sin remoción IQR")
    robustez_txt = " · ".join(robustez_bits)

    # --- Decisión e interpretación ---
    if sig and not cruza_cero:
        decision = f"**Evidencia estadísticamente significativa** (p={p:.2e} < α={alpha})."
        ic_read = f"El IC {int((1-alpha)*100)}% **no** cruza 0 ({lo:.2f}, {hi:.2f})."
        implicacion = (
            "La diferencia es consistente y **probablemente relevante** si el tamaño de efecto no es trivial."
            if mag not in ("trivial",) else
            "La diferencia es consistente pero su **magnitud práctica parece limitada** (efecto trivial)."
        )
    elif sig and cruza_cero:
        # caso raro: p<alpha pero IC percentil cruza 0 (discrepancia param/boot)
        decision = f"**Resultado mixto**: p={p:.2e} < α, pero el IC bootstrap cruza 0."
        ic_read = f"El IC {int((1-alpha)*100)}% **cruza 0** ({lo:.2f}, {hi:.2f})."
        implicacion = "Sugerimos validar con más réplicas bootstrap y/o tamaño muestral mayor."
    else:
        decision = f"**No se detecta diferencia estadísticamente significativa** (p={p:.2e} ≥ α={alpha})."
        ic_read = f"El IC {int((1-alpha)*100)}% incluye 0 ({lo:.2f}, {hi:.2f})."
        implicacion = "Con la evidencia actual, la diferencia podría ser nula o demasiado pequeña para detectarse."

    # --- Recomendaciones accionables (según signo/magnitud) ---
    if sig and abs(eff) >= 0.3:
        # umbral medio para recomendar acción si hay señal consistente
        rec = (
            "- **Operación/planeación**: ajustar buffers o expectativas de servicio en días lluviosos.\n"
            "- **Comunicación**: anticipar al usuario/clientes posibles demoras bajo lluvia.\n"
            "- **Seguimiento**: monitorear este gap en próximos meses para validar estabilidad."
        )
    elif sig and abs(eff) < 0.3:
        rec = (
            "- La diferencia es **pequeña**; considerar acciones **puntuales** (p. ej., microajustes de programación) "
            "solo si el costo de intervención es bajo."
        )
    else:
        rec = (
            "- **No accionar cambios** por ahora. Priorizar **más datos** o analizar subgrupos (hora pico, temporada) "
            "para descartar efectos locales."
        )

    # --- Ensamble Markdown
    md = f"""
### Conclusiones — {ruta}: *lluvioso vs sin lluvia*

**Prueba**: {test}  
**p-valor**: {p:.2e} · **α** = {alpha}  
**Diferencia en {kpi}** (lluvioso − sin lluvia): **{dif:+.2f} min{pct_txt}**  
**IC {int((1-alpha)*100)}%**: [{lo:.2f}, {hi:.2f}] min  
**Tamaño de efecto**: {eff_txt}  _(guías: {guias_txt})_  
**Tamaño muestral**: lluvioso **n={n_ll}**, sin lluvia **n={n_sl}**  
**Robustez**: {robustez_txt}

> **Lectura**: {sentido_txt}. {decision} {ic_read}

**Implicación práctica**  
{implicacion}

**Siguientes pasos / Recomendaciones**  
{rec}
""".strip()

    display(Markdown(md))

# Renderizar conclusiones en Markdown (usa los flags globales definidos previamente)
conclusiones_markdown(resultados_prueba, alpha=ALFA, contexto={
    'rm_outliers': RM_OUT,
    'B': NBOOT,
    'seed': RNG_SEED,
    'baseline_ref_minutes': df_limpio.loc[df_limpio['clima'].eq('sin lluvia'), 'duracion_s'].mean() / 60,
    'kpi': 'duración (min)',
    'ruta': "Loop -> O'Hare"
})

## OJO:

**¿Por qué α = 0.05?**  
- Es un estándar ampliamente aceptado que equilibra **riesgo de falso positivo** (Tipo I) y **potencia**.  
- **Cuándo cambiarlo**:  
  - **α = 0.10** si prima la **sensibilidad** (prefieres detectar diferencias pequeñas aunque aumente el riesgo de falso positivo).  
  - **α = 0.01** si las decisiones son **críticas** (p. ej., alto costo por actuar erróneamente) o hay **múltiples comparaciones** (tras Bonferroni/Benjamini–Hochberg).  
  - **Pre-especifica** α antes de mirar los datos para evitar sesgos.
